In [ ]:
%%capture
%load_ext sql
%config SqlMagic.feedback = False
%config SqlMagic.displaylimit = 0
%config SqlMagic.displaycon = False

In [ ]:
import datetime

start = datetime.datetime.now() - datetime.timedelta(days=90)
end = datetime.datetime.now()

In [ ]:
%%sql repeat_incident_locations <<

SELECT location_city_town, location_state, location_name, COUNT(*) count
FROM cts_forms_report
WHERE location_name notnull
AND location_city_town notnull
AND location_state notnull
GROUP BY location_name, location_city_town, location_state
HAVING COUNT(*) > 1
ORDER BY count desc

In [1]:
from IPython.display import display, HTML
import numpy as np

readable_start = start.strftime("%B %d, %Y")
readable_end = end.strftime("%B %d, %Y")
incident_locations = repeat_incident_locations.DataFrame()
cities = incident_locations['location_city_town'].values
location_citystate_divs = []
location_citystate_dropdown = []
location_name_divs = []
for i, city in enumerate(cities):
    state = incident_locations['location_state'].values[i]
    location_citystate_dropdown.append(
        f'<option value="{city}-{state}">{city}, {state}</option>'
    )
    location_name = incident_locations['location_name'].values[i]
    count = incident_locations['count'].values[i]
    location_name_divs.append(
        f"""<div class="flex-align-center city-wrapper {city}-{state} border-bottom border-base-light padding-top-1 padding-bottom-1 flex-row grid-row flex-justify display-flex">
                <strong class="grid-col-3">{location_name}</strong>
                <span class="grid-col-3">{city}, {state}</span>
                <a class="grid-col-3 flex-align-self-end" href="http://localhost:8000/form/view/?location_name={location_name}&location_city_town={city}&location_state={state}&grouping=default">
                    <strong>View {count} reports</strong>
                </a>
            </div>
            """
    )

display(HTML(f"""
    <div class="crt-portal-card light maxh-mobile-lg overflow-y-scroll">
    <h3>Repeated incident location names by city</h3>
    <select name="incident_location_city" class="usa-select incident-location-city" aria-label="Incident location city">
        <option value="">Select a city</option>
        {''.join(sorted(np.unique(location_citystate_dropdown)))}
    </select>
    <p style="font-size: 0.75rem">
      <em>Repeated report locations submitted between
        <strong>{readable_start}</strong> and
        <strong>{readable_end}</strong>
      </em>
    </p>
    <br />
     <div class="flex-align-center districts flex-row grid-row flex-justify display-flex">
        <strong class="grid-col-3">Location Name</strong>
        <strong class="grid-col-3">City, State</strong>
        <strong class="grid-col-3 flex-align-self-end"># of Reports</strong>
    </div>
    <hr />
    {''.join(sorted(location_name_divs))}
    </div>
"""))

ModuleNotFoundError: No module named 'numpy'